<a href="https://colab.research.google.com/github/jcandane/RCF/blob/main/gpflow_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf

try:
    import gpflow
except:
    !pip install gpflow
    import gpflow

import plotly.graph_objects as go

In [2]:
R_ix = tf.random.uniform( (5,4) , dtype=tf.float64)
D_ix = tf.random.uniform( (6,4) , dtype=tf.float64)

k    = gpflow.kernels.SquaredExponential()

kk   = gpflow.kernels.Matern12()

In [3]:
kk.K(R_ix, D_ix) ## compute covariance matrix

L_ij = tf.linalg.cholesky( kk.K(R_ix) ) ## tf cholesky

tf.linalg.cholesky_solve(L_ij, R_ix) ## tf.linalg.cholesky_solve(L_ij, RHS) R_ix is an example!!

<tf.Tensor: shape=(5, 4), dtype=float64, numpy=
array([[ 0.15456636,  0.41987092,  0.99264408,  0.14565326],
       [-0.0308443 ,  0.11688812, -0.04738799,  0.09308394],
       [ 0.51070729,  0.06289228, -0.45488517,  0.14346443],
       [ 0.55451982,  0.58663799,  0.24543161,  0.6064408 ],
       [-0.20549746,  0.23522289,  0.07849948, -0.4449282 ]])>

## gpFLOW RCF function

In [4]:
tf.keras.backend.set_floatx('float64')

Domain = tf.constant([[0,10.],[-3,4.]], dtype=tf.float64) #torch.tensor([[0,10.],[-3,4.],[-8,-2]]) ### numpy.2darray
N      = 32  ### number of defining points
MO     = 1   ### int (dimension of OUT)

kernel = gpflow.kernels.SquaredExponential() ##gpx.kernels.RBF()
seed   = 1372
############################

tf.random.set_seed( seed )
R_ix  = tf.random.uniform( ( N, Domain.shape[0]) , dtype=tf.float64)
R_ix  = (Domain[:,1] - Domain[:,0]) * R_ix
R_ix += Domain[:,0] ## save this!!!!!

L_ij = tf.linalg.cholesky( kernel.K(R_ix) ) ## tf cholesky

D_iX  = tf.random.normal((N, MO), dtype=tf.float64)
D_iX *= tf.linalg.diag_part(L_ij)
D_iX  = tf.matmul(L_ij, D_iX)

S_jX  = tf.linalg.cholesky_solve(L_ij, D_iX) ## save this!!!!!

In [5]:
#### generate mesh to plot
R_ax = tf.stack(tf.meshgrid(*[ tf.range(Domain[i,0], Domain[i,1], 0.33) for i in range(len(Domain)) ]), axis=-1)
#R_ax = R_ax.reshape((tf.prod( tf.asarray(R_ax.shape[:-1]) ), R_ax.shape[-1]))
shaper = tf.concat((tf.constant(R_ax.shape[:-1], dtype=tf.int64), R_ax.shape[-1]*tf.ones(1, dtype=tf.int64)), 0)
R_ax = tf.reshape(R_ax, (tf.math.reduce_prod(R_ax.shape[:-1]), R_ax.shape[-1]))

R_ay = (tf.matmul(kernel(R_ax, R_ix), S_jX)).numpy()
R_ax = R_ax.numpy()

#### the plot
fig = go.Figure(data=[go.Scatter3d(x=R_ax[:,0], y=R_ax[:,1], z=R_ay[:,0], mode='markers'),
                      go.Scatter3d(x=R_ix.numpy()[:,0], y=R_ix.numpy()[:,1], z=D_iX.numpy()[:,0], mode='markers')])
fig.show()

## class

In [6]:
import tensorflow as tf
import gpflow

class RCF():
    """ built: 3/19/2024
    this an object of a Random-Contionus-Function (RCF), with-respect-to a gpflow kernel
    RCF : IN -> OUT = R^(MO)
    we define a prior, and then sample to form a posterior.
    """

    def __init__(self, Domain:tf.Tensor, MO:int=1, N:int=17, seed:int=777,
                 IN_noise=None, OUT_noise=None,
                 kernel=gpflow.kernels.SquaredExponential()):
        """ !! note datatypes should be tf.float64 for stable Cholesky-operations
        GIVEN >
             Domain : 2d-tf.Tensor (with shape=(d,2), with d=# of dims )
                  N : int (number-of-defining-points)
                 MO : int (Multiple-Output Dimension)
             **seed : int
           **kernel : gpflow.kernels
         **IN_noise : 1d-tf.Tensor (len == Domain.shape[1])
        **OUT_noise : 1d-tf.Tensor (len == MO)
        GET   >
            None
        """

        self.dtype  = tf.float64 ## datatype= float64 for stable cholesky-factor
        self.IN     = tf.cast(Domain, self.dtype)  ### : tf.Tensor (IN-space range)
        self.N      = N      ### number of defining points
        self.MO     = MO     ### int (dimension of OUT)
        self.kernel = kernel
        self.seed = seed ### define pseudo-random seed

        tf.random.set_seed( self.seed )

        ### define anisotropic i.i.d white-noise
        if IN_noise is None:
            self.IN_noise=tf.zeros(self.IN.shape[0], dtype=self.dtype)
        else:
            self.IN_noise = IN_noise
        if OUT_noise is None:
            self.OUT_noise=tf.zeros(self.MO, dtype=self.dtype)
        else:
            self.OUT_noise = OUT_noise

        ### define IN-space defining-points
        self.R_ix  = tf.random.uniform( (self.N, self.IN.shape[0]) , dtype=self.dtype)
        self.R_ix *= (self.IN[:,1] - self.IN[:,0])
        self.R_ix += self.IN[:,0]

        ### compute cholesky-factorization
        L_ij = tf.linalg.cholesky( self.kernel.K(self.R_ix) )
        if tf.reduce_sum( tf.cast( tf.math.is_nan(L_ij) , tf.int32 ), [0,1] )==0:
            None
        else: ### if cholesky-factorization fails... add small random diagonal
            L_ij = tf.linalg.cholesky( self.kernel.K(self.R_ix) + tf.linalg.diag( tf.random.uniform( (self.N, ) , dtype=self.dtype) ) )

        ### compute OUT-space defining-points
        D_iX  = tf.random.normal((self.N, self.MO), dtype=self.dtype)
        D_iX *= tf.linalg.diag_part(L_ij)
        D_iX  = tf.matmul(L_ij, D_iX)

        ### compute (L \ D) used to interpolate arbtirary points
        self.S_iX  = tf.linalg.cholesky_solve(L_ij, D_iX)

    def __call__(self, D_ax):
        """ evaluate for arbitrary values/points in OUT given points in IN.
        GIVEN >
              self
              D_ax : 2d-tf.Tensor (D_ax ∈ IN)
        GET   >
              D_aX : 2d-tf.Tensor (D_aX ∈ OUT, note captial 'X')
        """
        D_ax += self.IN_noise*tf.random.normal(D_ax.shape, dtype=self.dtype)
        D_aX  = tf.matmul(self.kernel(D_ax, self.R_ix), self.S_iX)
        D_aX += self.OUT_noise*tf.random.normal(D_aX.shape, dtype=self.dtype)
        return D_aX

In [7]:
Domain = tf.constant([[0,10.],[-3,4.]], dtype=tf.float64) #torch.tensor([[0,10.],[-3,4.],[-8,-2]]) ### numpy.2darray


f = RCF(tf.cast(Domain, tf.float16), N=18, seed=48) ## problems seed=1287

#### generate mesh to plot
R_ax = tf.stack(tf.meshgrid(*[ tf.linspace(Domain[i,0], Domain[i,1], 20) for i in range(len(Domain)) ]), axis=-1)
#R_ax = R_ax.reshape((tf.prod( tf.asarray(R_ax.shape[:-1]) ), R_ax.shape[-1]))
shaper = tf.concat((tf.constant(R_ax.shape[:-1], dtype=tf.int64), R_ax.shape[-1]*tf.ones(1, dtype=tf.int64)), 0)
R_ax = tf.reshape(R_ax, (tf.math.reduce_prod(R_ax.shape[:-1]), R_ax.shape[-1]))

R_ay = f(R_ax).numpy()
R_ax = R_ax.numpy()

#### the plot
fig = go.Figure(data=[go.Scatter3d(x=R_ax[:,0], y=R_ax[:,1], z=R_ay[:,0], mode='markers'),
                      go.Scatter3d(x=f.R_ix.numpy()[:,0], y=f.R_ix.numpy()[:,1], z=f(f.R_ix).numpy()[:,0], mode='markers')])
fig.show()

## very small domain, hence close test

looks straight, because on the length-scale of the kernel, its essentially constant (a point).

!! tensorflow's Cholesky-factorization does not have the safety mechanism of pytorch

In [9]:
Domain = tf.constant([[0,1.e-2],[-1.e-2,1.e-2]], dtype=tf.float64) #torch.tensor([[0,10.],[-3,4.],[-8,-2]]) ### numpy.2darray


f = RCF(tf.cast(Domain, tf.float16), N=180, seed=481) ## problems seed=1287

#### generate mesh to plot
R_ax = tf.stack(tf.meshgrid(*[ tf.linspace(Domain[i,0], Domain[i,1], 20) for i in range(len(Domain)) ]), axis=-1)
#R_ax = R_ax.reshape((tf.prod( tf.asarray(R_ax.shape[:-1]) ), R_ax.shape[-1]))
shaper = tf.concat((tf.constant(R_ax.shape[:-1], dtype=tf.int64), R_ax.shape[-1]*tf.ones(1, dtype=tf.int64)), 0)
R_ax = tf.reshape(R_ax, (tf.math.reduce_prod(R_ax.shape[:-1]), R_ax.shape[-1]))

R_ay = f(R_ax).numpy()
R_ax = R_ax.numpy()

#### the plot
fig = go.Figure(data=[go.Scatter3d(x=R_ax[:,0], y=R_ax[:,1], z=R_ay[:,0], mode='markers'),
                      go.Scatter3d(x=f.R_ix.numpy()[:,0], y=f.R_ix.numpy()[:,1], z=f(f.R_ix).numpy()[:,0], mode='markers')])
fig.show()